# ASA Metadata Registry (ARC-89) SDK Demo

This notebook demonstrates all the capabilities of the ASA Metadata Registry Python SDK, using the **TestNet** deployment.

## Overview

The ASA Metadata Registry implements [ARC-89](https://github.com/cusma/ARCs/blob/arc89/ARCs/arc-0089.md), providing on-chain JSON metadata storage for Algorand Standard Assets (ASAs).

### Key Features Covered

1. **Reading Metadata**
  - Fast Algod box reads
  - AVM-parity reads via simulate
  - Metadata existence checks
  - Pagination handling

2. **Writing Metadata**
  - Creating metadata
  - Replacing metadata
  - Deleting metadata
  - Flag management
3. **SDK Helpers**
  - ARC-90 URI construction and parsing
  - Hash computation
  - MBR calculations
  - Metadata models and flags

## Setup

First, let's install the required dependencies and set up the environment.

In [1]:
# Install the SDK (if not already installed)
%pip install asa-metadata-registry python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables from .env.testnet in project root
project_root = Path.cwd().parent
load_dotenv(dotenv_path=project_root / ".env.testnet")

# Verify the environment variable is set (we'll use it later for write operations)
if os.getenv("CALLER_MNEMONIC"):
    print("✓ CALLER_MNEMONIC environment variable is set")
else:
    print("⚠ CALLER_MNEMONIC not set - write operations will fail")
    print("  Set CALLER_MNEMONIC in a .env.testnet file or export it in your shell")

✓ CALLER_MNEMONIC environment variable is set


## 1. Imports and ConstantsLet's import all the SDK components we'll use throughout this demo.

In [3]:
from algokit_utils import AlgorandClient, AssetCreateParams

# Main SDK imports
from asa_metadata_registry import (
    # Deployments
    DEFAULT_DEPLOYMENTS,
    Arc90Compliance,
    # Codec / ARC-90 URI
    Arc90Uri,
    # Facade and config
    AsaMetadataRegistry,
    # Hashing
    AsaNotFoundError,
    # Models
    AssetMetadata,
    IrreversibleFlags,
    MetadataFlags,
    MetadataSource,
    ReversibleFlags,
    WriteOptions,
    codec,
    complete_partial_asset_url,
    get_default_registry_params,
    is_arc3_metadata,
)

# Generated client (required for AVM reads and writes)
from asa_metadata_registry._generated.asa_metadata_registry_client import AsaMetadataRegistryClient

print("✓ All SDK components imported successfully")

✓ All SDK components imported successfully


In [4]:
# TestNet deployment info
testnet_deployment = DEFAULT_DEPLOYMENTS["testnet"]
print(f"Network: {testnet_deployment.network}")
print(f"App ID: {testnet_deployment.app_id}")
print(f"Creator Address: {testnet_deployment.creator_address}")
print(f"ARC-90 URI Netauth: {testnet_deployment.arc90_uri_netauth}")

Network: testnet
App ID: 753324084
Creator Address: QYK5DXJ27Y7WIWUJMP3FFOTEU56L4KTRP4CY2GAKRXZHHKLNWV6M7JLYJM
ARC-90 URI Netauth: net:testnet


## 2. ARC-90 URI Helpers

ARC-90 defines the URI format for referencing on-chain metadata. The SDK provides comprehensive tools for building and parsing these URIs.

### 2.1 Building ARC-90 URIs

In [5]:
# Create a partial URI (used in ASA's url field - box value is empty)
partial_uri = Arc90Uri(
    netauth=testnet_deployment.arc90_uri_netauth,
    app_id=testnet_deployment.app_id,
    box_name=None,  # Partial - no box value yet
)

print("Partial URI (for ASA url field):")
print(f"  {partial_uri.to_uri()}")
print(f"  Is partial: {partial_uri.is_partial}")

Partial URI (for ASA url field):
  algorand://net:testnet/app/753324084?box=
  Is partial: True


In [6]:
# Complete the URI with an asset ID
example_asset_id = int(os.environ["ARC3_NFT_ID"])  # An existing TestNet asset with metadata
complete_uri = partial_uri.with_asset_id(example_asset_id)

print("Complete URI (full reference to metadata):")
print(f"  {complete_uri.to_uri()}")
print(f"  Is partial: {complete_uri.is_partial}")
print(f"  Asset ID: {complete_uri.asset_id}")
print(f"  App ID: {complete_uri.app_id}")

Complete URI (full reference to metadata):
  algorand://net:testnet/app/753324084?box=AAAAACzm0D8%3D
  Is partial: False
  Asset ID: 753324095
  App ID: 753324084


In [7]:
# Add ARC compliance fragment
uri_with_compliance = Arc90Uri(
    netauth=testnet_deployment.arc90_uri_netauth,
    app_id=testnet_deployment.app_id,
    box_name=None,
    compliance=Arc90Compliance(arcs=(20, 62)),  # ARC-20 and ARC-62 compliant
)

print("URI with compliance fragment:")
print(f"  {uri_with_compliance.to_uri()}")

URI with compliance fragment:
  algorand://net:testnet/app/753324084?box=#arc20+62


### 2.2 Parsing ARC-90 URIs

In [8]:
# Parse a complete URI
uri_string = "algorand://net:testnet/app/753324084?box=AAAAAALNgR8=#arc89"
parsed = Arc90Uri.parse(uri_string)

print("Parsed URI components:")
print(f"  Netauth: {parsed.netauth}")
print(f"  App ID: {parsed.app_id}")
print(f"  Asset ID: {parsed.asset_id}")
print(f"  Box name (hex): {parsed.box_name.hex() if parsed.box_name else None}")
print(f"  Compliance ARCs: {parsed.compliance.arcs}")

Parsed URI components:
  Netauth: net:testnet
  App ID: 753324084
  Asset ID: 47022367
  Box name (hex): 0000000002cd811f
  Compliance ARCs: (89,)


In [9]:
# Complete a partial ASA URL
partial_asa_url = "algorand://net:testnet/app/753324084?box=#arc89"
asset_id = 12345678
completed_url = complete_partial_asset_url(partial_asa_url, asset_id)

print(f"Original partial URL: {partial_asa_url}")
print(f"Completed URL: {completed_url}")

Original partial URL: algorand://net:testnet/app/753324084?box=#arc89
Completed URL: algorand://net:testnet/app/753324084?box=AAAAAAC8YU4%3D#arc89


## 3. Reading Metadata

The SDK supports two reading modes:

- **BOX mode**: Fast direct Algod box reads (1 request)
- **AVM mode**: Smart contract getter calls via simulate (AVM-parity)

### 3.1 Algod Client Setup (Read-Only)

In [10]:
# Create an Algorand client for TestNet
algorand_client = AlgorandClient.testnet()
algorand_client.set_default_validity_window(100)

# Create a read-only registry (no signer needed)
registry_readonly = AsaMetadataRegistry.from_algod(
    algod=algorand_client.client.algod,
    app_id=testnet_deployment.app_id,
)

print(f"✓ Read-only registry created for app {testnet_deployment.app_id}")

✓ Read-only registry created for app 753324084


### 3.2 Check Metadata Existence

In [11]:
# Check if metadata exists for an asset
test_asset_id = int(os.environ["ARC3_NFT_ID"])  # Known TestNet asset with metadata
existence = registry_readonly.read.arc89_check_metadata_exists(
    asset_id=test_asset_id,
    source=MetadataSource.BOX,
)

print(f"Asset {test_asset_id}:")
print(f"  ASA exists: {existence.asa_exists}")
print(f"  Metadata exists: {existence.metadata_exists}")

Asset 753324095:
  ASA exists: True
  Metadata exists: True


In [12]:
# Check a non-existent asset
fake_asset_id = 999999999999
existence = registry_readonly.read.arc89_check_metadata_exists(
    asset_id=fake_asset_id,
    source=MetadataSource.BOX,
)

print(f"Asset {fake_asset_id}:")
print(f"  ASA exists: {existence.asa_exists}")
print(f"  Metadata exists: {existence.metadata_exists}")

Asset 999999999999:
  ASA exists: False
  Metadata exists: False


### 3.3 Reading Full Metadata (BOX Mode)

In [13]:
# Read complete metadata record using fast BOX mode
record = registry_readonly.read.get_asset_metadata(
    asset_id=test_asset_id,
    source=MetadataSource.BOX,
)

print(f"Asset Metadata Record for {test_asset_id}:")
print(f"  App ID: {record.app_id}")
print(f"  Asset ID: {record.asset_id}")
print(f"  Metadata size: {record.body.size} bytes")
print(f"  Is short: {record.body.is_short}")
print(f"  Is empty: {record.body.is_empty}")

Asset Metadata Record for 753324095:
  App ID: 753324084
  Asset ID: 753324095
  Metadata size: 368 bytes
  Is short: True
  Is empty: False


In [14]:
# Access the metadata header
header = record.header

print("Metadata Header:")
print(f"  Identifiers byte: {header.identifiers:#04x}")
print(f"  Is short (from identifiers): {header.is_short}")
print(f"  Is immutable: {header.is_immutable}")
print(f"  Is ARC-3 compliant: {header.is_arc3_compliant}")
print(f"  Is ARC-89 native: {header.is_arc89_native}")
print(f"  Is deprecated: {header.is_deprecated}")
print(f"  Last modified round: {header.last_modified_round}")
print(f"  Metadata hash: {header.metadata_hash.hex()[:32]}...")

Metadata Header:
  Identifiers byte: 0x80
  Is short (from identifiers): True
  Is immutable: True
  Is ARC-3 compliant: True
  Is ARC-89 native: True
  Is deprecated: False
  Last modified round: 59305318
  Metadata hash: 1b08c49fad7d4b3556399a43553e4d05...


In [15]:
# Access the metadata as JSON
import json

metadata_json = record.json

print("Metadata JSON:")
print(json.dumps(metadata_json, indent=2))

Metadata JSON:
{
  "name": "Cusma #2",
  "description": "Cusma autograph #2",
  "image": "ipfs://bafkreigxshpznrgo5srv3oqegapqez6hpwkiwvyckvlxpbobtrzq3jyzta",
  "decimals": 0,
  "unitName": "CUSMA",
  "image_integrity": "sha256-15HflsTO7KNdugQwHwJnx32Ui1cCVVd3hcGccw2nGZg=",
  "image_mimetype": "image/png",
  "properties": {
    "background_color_hex": "#FFFFFF",
    "writing_color_hex": "#285FF4",
    "to": "Fred",
    "size": 589362
  }
}


### 3.4 Reading Individual Components

In [16]:
# Read just the header (faster if you don't need the full body)
header_only = registry_readonly.read.arc89_get_metadata_header(
    asset_id=test_asset_id,
    source=MetadataSource.BOX,
)

print("Header (standalone read):")
print(f"  Metadata hash: {header_only.metadata_hash.hex()[:32]}...")

Header (standalone read):
  Metadata hash: 1b08c49fad7d4b3556399a43553e4d05...


In [17]:
# Check if metadata is short (<=512 bytes)
is_short, metadata_size = registry_readonly.read.arc89_is_metadata_short(
    asset_id=test_asset_id,
    source=MetadataSource.BOX,
)

print("Metadata size check:")
print(f"  Is short: {is_short}")
print(f"  Size: {metadata_size} bytes")

Metadata size check:
  Is short: True
  Size: 59305318 bytes


In [18]:
# Check immutability
is_immutable = registry_readonly.read.arc89_is_metadata_immutable(
    asset_id=test_asset_id,
    source=MetadataSource.BOX,
)

print(f"Is metadata immutable: {is_immutable}")

Is metadata immutable: True


### 3.5 AVM-Parity Reads (via Simulate)For AVM reads, we need an App Client with a sender configured.

In [19]:
# Create a full registry with App Client for AVM reads
# This requires a sender account (can be any account for reads)
try:
    caller = algorand_client.account.from_environment(name="CALLER")

    testnet_app_client = algorand_client.client.get_typed_app_client_by_id(
        AsaMetadataRegistryClient,
        app_id=testnet_deployment.app_id,
        default_sender=caller.address,
        default_signer=caller.signer,
    )

    registry_full = AsaMetadataRegistry.from_app_client(
        testnet_app_client,
        algod=algorand_client.client.algod,  # Also enable BOX reads
    )

    print(f"Full registry created with sender: {caller.address[:20]}...")
    HAS_CALLER = True
except Exception as e:
    print(f"Could not create full registry: {e}")
    print("  AVM reads and writes will be skipped")
    HAS_CALLER = False

Full registry created with sender: OLB2FETIUSZ5XTOJ7AYU...


In [20]:
if HAS_CALLER:
    # Get registry parameters via AVM call
    params = registry_full.read.arc89_get_metadata_registry_parameters(
        source=MetadataSource.AVM,
    )

    print("Registry Parameters (from on-chain):")
    print(f"  Key size: {params.key_size} bytes")
    print(f"  Header size: {params.header_size} bytes")
    print(f"  Max metadata size: {params.max_metadata_size} bytes")
    print(f"  Short metadata threshold: {params.short_metadata_size} bytes")
    print(f"  Page size: {params.page_size} bytes")
    print(f"  First payload max: {params.first_payload_max_size} bytes")
    print(f"  Extra payload max: {params.extra_payload_max_size} bytes")
    print(f"  Flat MBR: {params.flat_mbr} microALGO")
    print(f"  Byte MBR: {params.byte_mbr} microALGO/byte")

Registry Parameters (from on-chain):
  Key size: 8 bytes
  Header size: 51 bytes
  Max metadata size: 30506 bytes
  Short metadata threshold: 4096 bytes
  Page size: 1007 bytes
  First payload max: 2030 bytes
  Extra payload max: 2034 bytes
  Flat MBR: 2500 microALGO
  Byte MBR: 400 microALGO/byte


In [21]:
if HAS_CALLER:
    # Get the partial URI from the contract
    partial_uri_onchain = registry_full.read.arc89_get_metadata_partial_uri(
        source=MetadataSource.AVM,
    )

    print(f"Partial URI from contract: {partial_uri_onchain}")

Partial URI from contract: algorand://net:testnet/app/753324084?box=


In [22]:
if HAS_CALLER:
    # Read metadata via AVM (uses simulate)
    record_avm = registry_full.read.get_asset_metadata(
        asset_id=test_asset_id,
        source=MetadataSource.AVM,
    )

    print("Metadata via AVM:")
    print(f"  Size: {record_avm.body.size} bytes")
    print(f"  Hash matches BOX read: {record_avm.header.metadata_hash == record.header.metadata_hash}")

Metadata via AVM:
  Size: 368 bytes
  Hash matches BOX read: True


### 3.6 Direct Box and AVM Sub-Readers

In [23]:
# Access the box reader directly
box_reader = registry_readonly.read.box

# Get raw box data
box = box_reader.get_asset_metadata_record(asset_id=test_asset_id)

print("Box data:")
print(f"  Asset ID: {box.asset_id}")
print(f"  Header size: {len(box.header.serialized)} bytes")
print(f"  Body size: {box.body.size} bytes")

Box data:
  Asset ID: 753324095
  Header size: 51 bytes
  Body size: 368 bytes


In [24]:
if HAS_CALLER:
    # Access the AVM reader directly
    avm_reader = registry_full.read.avm()

    # Read individual pages
    page0 = avm_reader.arc89_get_metadata(asset_id=test_asset_id, page=0)

    print("Page 0:")
    print(f"  Has next page: {page0.has_next_page}")
    print(f"  Last modified round: {page0.last_modified_round}")
    print(f"  Content size: {len(page0.page_content)} bytes")

Page 0:
  Has next page: False
  Last modified round: 59305318
  Content size: 368 bytes


## 4. Metadata Models and Helpers

The SDK provides rich model classes for working with metadata.

### 4.1 Creating Metadata from JSON

In [25]:
# Create metadata for a hypothetical asset
example_metadata = AssetMetadata.from_json(
    asset_id=12345678,
    json_obj={
        "name": "Demo Token",
        "description": "A demonstration token for the SDK demo",
        "properties": {
            "rarity": "legendary",
            "power": 9000,
        },
    },
)

print("Example Metadata:")
print(f"  Asset ID: {example_metadata.asset_id}")
print(f"  Size: {example_metadata.size} bytes")
print(f"  Is short: {example_metadata.is_short}")
print(f"  Is empty: {example_metadata.is_empty}")

Example Metadata:
  Asset ID: 12345678
  Size: 125 bytes
  Is short: True
  Is empty: False


In [26]:
# Create ARC-3 compliant metadata (will be auto-detected by is_arc3_metadata)
arc3_json_obj = {
    "name": "ARC-3 Compliant NFT",
    "description": "An NFT following the ARC-3 standard",
    "image": "ipfs://QmExample...",
    "image_mimetype": "image/png",
    "properties": {
        "artist": "Anonymous",
        "collection": "Demo Collection",
    },
}

arc3_metadata = AssetMetadata.from_json(
    asset_id=12345678,
    json_obj=arc3_json_obj,
    arc3_compliant=is_arc3_metadata(arc3_json_obj),
)

print("ARC-3 Metadata:")
print(f"  Is ARC-3 compliant (flag): {arc3_metadata.is_arc3_compliant}")
print(f"  Reversible Flags: {arc3_metadata.flags.reversible_byte:#04x}")
print(f"  Irreversible Flags: {arc3_metadata.flags.irreversible_byte:#04x}")

ARC-3 Metadata:
  Is ARC-3 compliant (flag): True
  Reversible Flags: 0x00
  Irreversible Flags: 0x01


### 4.2 Working with Flags

In [27]:
# Create custom flags
custom_flags = MetadataFlags(
    reversible=ReversibleFlags(
        arc20=True,   # Smart ASA
        arc62=False,  # No circulating supply tracking
    ),
    irreversible=IrreversibleFlags(
        arc3=True,          # ARC-3 compliant
        arc89_native=True,  # Native ARC-89 metadata
        immutable=False,    # Can be updated
    ),
)

print("Custom Flags:")
print(f"  Reversible byte: {custom_flags.reversible_byte:#04x} ({custom_flags.reversible_byte:08b})")
print(f"  Irreversible byte: {custom_flags.irreversible_byte:#04x} ({custom_flags.irreversible_byte:08b})")
print(f"  ARC-20 (Smart ASA): {custom_flags.reversible.arc20}")
print(f"  ARC-62 (Circulating Supply): {custom_flags.reversible.arc62}")
print(f"  ARC-3 Compliant: {custom_flags.irreversible.arc3}")
print(f"  ARC-89 Native: {custom_flags.irreversible.arc89_native}")
print(f"  Immutable: {custom_flags.irreversible.immutable}")

Custom Flags:
  Reversible byte: 0x01 (00000001)
  Irreversible byte: 0x03 (00000011)
  ARC-20 (Smart ASA): True
  ARC-62 (Circulating Supply): False
  ARC-3 Compliant: True
  ARC-89 Native: True
  Immutable: False


In [28]:
# Parse flags from bytes (e.g., from on-chain data)
rev_byte = 0b00000011  # ARC-20 + ARC-62
irr_byte = 0b10000001  # ARC-3 + Immutable
parsed_flags = MetadataFlags.from_bytes(rev_byte, irr_byte)

print("Parsed Flags:")
print(f"  ARC-20 (Smart ASA): {parsed_flags.reversible.arc20}")
print(f"  ARC-62 (Circulating Supply): {parsed_flags.reversible.arc62}")
print(f"  ARC-3: {parsed_flags.irreversible.arc3}")
print(f"  Immutable: {parsed_flags.irreversible.immutable}")

Parsed Flags:
  ARC-20 (Smart ASA): True
  ARC-62 (Circulating Supply): True
  ARC-3: True
  Immutable: True


### 4.3 MBR (Minimum Balance Requirement) Calculations

In [29]:
# Get default registry parameters
params = get_default_registry_params()

# Calculate MBR for different metadata sizes
sizes = [0, 100, 512, 1000, 8000]

print("MBR Requirements by Metadata Size:")
print("-" * 40)
for size in sizes:
    mbr = params.mbr_for_box(size)
    print(f"  {size:5} bytes -> {mbr:,} microALGO ({mbr/1_000_000:.4f} ALGO)")

MBR Requirements by Metadata Size:
----------------------------------------
      0 bytes -> 26,100 microALGO (0.0261 ALGO)
    100 bytes -> 66,100 microALGO (0.0661 ALGO)
    512 bytes -> 230,900 microALGO (0.2309 ALGO)
   1000 bytes -> 426,100 microALGO (0.4261 ALGO)
   8000 bytes -> 3,226,100 microALGO (3.2261 ALGO)


In [30]:
# Calculate MBR delta for creating metadata
new_size = 500
mbr_delta = params.mbr_delta(old_metadata_size=None, new_metadata_size=new_size)

print(f"Creating {new_size} bytes of metadata:")
print(f"  MBR delta: {mbr_delta.signed_amount:+,} microALGO")
print(f"  Is positive: {mbr_delta.is_positive}")

Creating 500 bytes of metadata:
  MBR delta: +226,100 microALGO
  Is positive: True


In [31]:
# Calculate MBR delta for replacing metadata (smaller -> larger)
mbr_delta_grow = params.mbr_delta(old_metadata_size=500, new_metadata_size=1000)
mbr_delta_shrink = params.mbr_delta(old_metadata_size=1000, new_metadata_size=500)

print("Replacing 500 -> 1000 bytes:")
print(f"  MBR delta: {mbr_delta_grow.signed_amount:+,} microALGO")
print("Replacing 1000 -> 500 bytes:")
print(f"  MBR delta: {mbr_delta_shrink.signed_amount:+,} microALGO")

Replacing 500 -> 1000 bytes:
  MBR delta: +200,000 microALGO
Replacing 1000 -> 500 bytes:
  MBR delta: -200,000 microALGO


In [32]:
# Calculate MBR delta for deleting metadata
mbr_delta_delete = params.mbr_delta(old_metadata_size=500, new_metadata_size=0, delete=True)

print("Deleting 500 bytes of metadata:")
print(f"  MBR refund: {mbr_delta_delete.signed_amount:+,} microALGO")
print(f"  Is negative: {mbr_delta_delete.is_negative}")

Deleting 500 bytes of metadata:
  MBR refund: -226,100 microALGO
  Is negative: True


### 4.4 Hash Computation

In [33]:
# Compute header hash
header_hash = example_metadata.compute_header_hash()
print(f"Header hash: {header_hash.hex()[:32]}...")

# Compute page hash
page_hash = example_metadata.compute_page_hash(page_index=0)
print(f"Page 0 hash: {page_hash.hex()[:32]}...")

# Compute full metadata hash (ARC-89 algorithm)
metadata_hash = example_metadata.compute_arc89_metadata_hash()
print(f"Metadata hash: {metadata_hash.hex()[:32]}...")

Header hash: 95cda879fa98e0c58d09f04f979a108f...
Page 0 hash: 33b8de3248b0dffd6bdc8f1379e3f200...
Metadata hash: 16370c85ff67360ae085fe1472ef5542...


In [34]:
# Verify hash matches on-chain data
computed_hash = record.expected_metadata_hash()
on_chain_hash = record.header.metadata_hash

asa_metadata_hash = codec.b64_decode(algorand_client.asset.get_by_id(test_asset_id).metadata_hash)

print(f"Hash verification for asset {test_asset_id}:")
print(f"  Computed: {computed_hash.hex()[:32]}...")
print(f"  On-chain: {on_chain_hash.hex()[:32]}...")
print(f"  ASA Metadata Hash: {asa_metadata_hash.hex()[:32]}...")
print(f"  Match: {record.hash_matches(asa_am=asa_metadata_hash)}")

Hash verification for asset 753324095:
  Computed: 07eb52221ea47aebc4fd36b6a101f2c7...
  On-chain: 1b08c49fad7d4b3556399a43553e4d05...
  ASA Metadata Hash: 1b08c49fad7d4b3556399a43553e4d05...
  Match: True


## 5. Writing Metadata

Write operations require an App Client with a signer. The asset manager must be the sender.

### 5.1 Creating a New Asset with Metadata

In [35]:
if HAS_CALLER:  # Step 1: Create the partial ARC-90 URI for the ASA's url field
    arc89_partial_uri = Arc90Uri(
        netauth=testnet_deployment.arc90_uri_netauth,
        app_id=testnet_app_client.app_id,
        box_name=None,
    ).to_uri()

    print(f"Partial URI for ASA: {arc89_partial_uri}")

Partial URI for ASA: algorand://net:testnet/app/753324084?box=


In [36]:
if HAS_CALLER:  # Step 2: Create the ASA with the partial URI
    result = algorand_client.send.asset_create(
        params=AssetCreateParams(
            sender=caller.address,
            total=1,
            asset_name="SDK Demo Token",
            unit_name="DEMO",
            url=arc89_partial_uri,
            decimals=0,
            manager=caller.address,  # Required to create metadata!
        )
    )
    new_asset_id = result.asset_id

    print(f"Created ASA with ID: {new_asset_id}")

Created ASA with ID: 753351402


In [37]:
if HAS_CALLER:  # Step 3: Prepare the metadata
    new_metadata = AssetMetadata.from_json(
        asset_id=new_asset_id,
        json_obj={
            "name": "SDK Demo Token",
            "description": "Created by the ASA Metadata Registry SDK demo notebook",
            "properties": {
                "created_at": "2026-01-09",
                "demo": True,
            },
        },
    )

    print("Prepared metadata:")
    print(f"  Size: {new_metadata.size} bytes")
    print(f"  Estimated MBR: {new_metadata.get_mbr_delta().amount:,} microALGO")

Prepared metadata:
  Size: 149 bytes
  Estimated MBR: 85,700 microALGO


In [38]:
if HAS_CALLER:
    # Step 4: Create the metadata on-chain
    mbr_result = registry_full.write.create_metadata(
        asset_manager=caller,
        metadata=new_metadata,
    )

    print("Metadata created!")
    print(f"  MBR paid: {mbr_result.amount:,} microALGO")
    print(f"  MBR direction: {'positive' if mbr_result.is_positive else 'refund'}")

Metadata created!
  MBR paid: 85,700 microALGO
  MBR direction: positive


In [39]:
if HAS_CALLER:
    # Step 5: Verify by reading it back
    read_back = registry_full.read.get_asset_metadata(
        asset_id=new_asset_id,
        source=MetadataSource.BOX,
    )

    print("Metadata read back:")
    print(json.dumps(read_back.json, indent=2))

Metadata read back:
{
  "name": "SDK Demo Token",
  "description": "Created by the ASA Metadata Registry SDK demo notebook",
  "properties": {
    "created_at": "2026-01-09",
    "demo": true
  }
}


### 5.2 Replacing Metadata

In [40]:
if HAS_CALLER:  # Prepare updated metadata
    updated_metadata = AssetMetadata.from_json(
        asset_id=new_asset_id,
        json_obj={
            "name": "SDK Demo Token (Updated)",
            "description": "This metadata was updated via the SDK",
            "properties": {
                "created_at": "2026-01-09",
                "updated_at": "2026-01-09",
                "demo": True,
                "version": 2,
            },
        },
    )

    print(f"Updated metadata size: {updated_metadata.size} bytes (was {new_metadata.size})")

Updated metadata size: 180 bytes (was 149)


In [41]:
if HAS_CALLER:  # Replace the metadata
    mbr_result = registry_full.write.replace_metadata(
        asset_manager=caller,
        metadata=updated_metadata,
    )

    print("Metadata replaced!")
    print(f"  MBR delta: {mbr_result.signed_amount:+,} microALGO")

Metadata replaced!
  MBR delta: +12,400 microALGO


In [42]:
if HAS_CALLER:  # Verify the update
    read_back = registry_full.read.get_asset_metadata(
        asset_id=new_asset_id,
        source=MetadataSource.BOX,
    )

    print("Updated metadata:")
    print(json.dumps(read_back.json, indent=2))

Updated metadata:
{
  "name": "SDK Demo Token (Updated)",
  "description": "This metadata was updated via the SDK",
  "properties": {
    "created_at": "2026-01-09",
    "updated_at": "2026-01-09",
    "demo": true,
    "version": 2
  }
}


### 5.3 Managing Reversible Flags

In [43]:
if HAS_CALLER:  # Check current flags
    header = registry_full.read.arc89_get_metadata_header(
        asset_id=new_asset_id,
        source=MetadataSource.BOX,
    )

    print("Current flags:")
    print(f"  ARC-20 (Smart ASA): {header.flags.reversible.arc20}")
    print(f"  ARC-62 (Circulating Supply): {header.flags.reversible.arc62}")

Current flags:
  ARC-20 (Smart ASA): False
  ARC-62 (Circulating Supply): False


In [44]:
if HAS_CALLER:
    from asa_metadata_registry import flags

    # Set the ARC-62 flag
    registry_full.write.set_reversible_flag(
        asset_manager=caller,
        asset_id=new_asset_id,
        flag_index=flags.REV_FLG_ARC62,  # Index 1
        value=True,
    )
    print("ARC-62 flag set to True")

    # Verify
    header = registry_full.read.arc89_get_metadata_header(
        asset_id=new_asset_id,
        source=MetadataSource.BOX,
    )
    print(f"  ARC-62 now: {header.flags.reversible.arc62}")

ARC-62 flag set to True
  ARC-62 now: True


### 5.4 Deleting Metadata

In [45]:
if HAS_CALLER:  # Delete the metadata (and get MBR refund)
    mbr_result = registry_full.write.delete_metadata(
        asset_manager=caller,
        asset_id=new_asset_id,
    )

    print("Metadata deleted!")
    print(f"  MBR refunded: {abs(mbr_result.signed_amount):,} microALGO")

Metadata deleted!
  MBR refunded: 98,100 microALGO


In [46]:
if HAS_CALLER:  # Verify deletion
    existence = registry_full.read.arc89_check_metadata_exists(
        asset_id=new_asset_id,
        source=MetadataSource.BOX,
    )

    print("After deletion:")
    print(f"  ASA exists: {existence.asa_exists}")
    print(f"  Metadata exists: {existence.metadata_exists}")

After deletion:
  ASA exists: True
  Metadata exists: False


## 6. Advanced Features

### 6.1 Building the ARC-90 URI from Registry

In [47]:
if HAS_CALLER:  # Use the registry's arc90_uri helper
    uri = registry_full.arc90_uri(asset_id=test_asset_id)

    print("ARC-90 URI via registry helper:")
    print(f"  {uri.to_uri()}")

ARC-90 URI via registry helper:
  algorand://app/753324084?box=AAAAACzm0D8%3D


### 6.2 Resolving URIs from Asset Info

In [48]:
# Resolve URI from asset ID (will check ASA's url field or use configured app_id)
resolved_uri = registry_readonly.read.resolve_arc90_uri(
    asset_id=test_asset_id,
)

print(f"Resolved URI: {resolved_uri.to_uri()}")

Resolved URI: algorand://net:testnet/app/753324084?box=AAAAACzm0D8%3D#arc3


### 6.3 Pagination for Large Metadata

In [49]:
# Calculate how large metadata would be paginated
params = get_default_registry_params()

# Create a large metadata body
large_json = {
    "name": "Large Metadata Example",
    "description": "This is a large metadata object" + " with lots of content" * 100,
    "properties": {
        f"prop_{i}": f"value_{i}" for i in range(50)
    },
}

large_metadata = AssetMetadata.from_json(
    asset_id=12345,
    json_obj=large_json,
)

print("Large metadata:")
print(f"  Size: {large_metadata.size:,} bytes")
print(f"  Total pages: {large_metadata.body.total_pages()}")
print(f"  Page size: {params.page_size} bytes")

Large metadata:
  Size: 3,226 bytes
  Total pages: 4
  Page size: 1007 bytes


In [50]:
# Access individual pages
for i in range(min(3, large_metadata.body.total_pages())):
    page = large_metadata.body.get_page(i)
    print(f"Page {i}: {len(page)} bytes")

Page 0: 1007 bytes
Page 1: 1007 bytes
Page 2: 1007 bytes


### 6.4 Metadata Chunking for Writes

In [51]:
# See how metadata would be chunked for write operations
chunks = large_metadata.body.chunked_payload()

print("Write payload chunking:")
print(f"  Total chunks: {len(chunks)}")
print(f"  First chunk (head): {len(chunks[0])} bytes (max {params.first_payload_max_size})")

if len(chunks) > 1:
    print(f"  Extra chunks: {[len(c) for c in chunks[1:]]} bytes each (max {params.extra_payload_max_size})")

Write payload chunking:
  Total chunks: 2
  First chunk (head): 2030 bytes (max 2030)
  Extra chunks: [1196] bytes each (max 2034)


### 6.5 Write Options

In [52]:
# Customize write behavior
write_options = WriteOptions(
    extra_resources=2,  # Add extra resource budget
    fee_padding_txns=1,  # Add fee padding for safety
    cover_app_call_inner_transaction_fees=True,  # Cover inner txn fees
)

print("Write options:")
print(f"  Extra resources: {write_options.extra_resources}")
print(f"  Fee padding: {write_options.fee_padding_txns}")
print(f"  Cover inner fees: {write_options.cover_app_call_inner_transaction_fees}")

Write options:
  Extra resources: 2
  Fee padding: 1
  Cover inner fees: True


## 7. Error Handling

In [53]:
from asa_metadata_registry import (
    InvalidArc90UriError,
)

# Try to read non-existent metadata
try:
  registry_readonly.read.get_asset_metadata(
      asset_id=999999999999,
      source=MetadataSource.BOX,
  )
except AsaNotFoundError as e:
    print(f"BoxNotFoundError caught: {e}")

BoxNotFoundError caught: ASA 999999999999 not found


In [54]:
# Try to parse an invalid URI
try:
    Arc90Uri.parse("https://example.com/not-an-arc90-uri")
except InvalidArc90UriError as e:
    print(f"InvalidArc90UriError caught: {e}")

InvalidArc90UriError caught: Not an algorand:// URI


## Summary

This notebook demonstrated the full capabilities of the ASA Metadata Registry SDK:

### Reading

- Fast Algod box reads (MetadataSource.BOX)
- AVM-parity reads via simulate (MetadataSource.AVM)
- Existence checks- Header and body access
- Pagination handling

### Writing

- Create metadata
- Replace metadata (handles size changes automatically)
- Delete metadata (with MBR refund)
- Manage reversible flags

### Helpers

- ARC-90 URI construction and parsing
- Partial URI completion
- Hash computation (header, page, full metadata)
- MBR calculations
- Metadata models and flags
- Write options customization

For more information, see:
- [ARC-89 Specification](https://github.com/cusma/ARCs/blob/arc89/ARCs/arc-0089.md)
- [SDK Repository](https://github.com/algorandfoundation/arc89)